<a href="https://colab.research.google.com/github/leobioinf0/BeRep/blob/main/BeRep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Dependencies

In [1]:
import pandas as pd, numpy as np, itertools

## Simple ten-liner

In [3]:
df = pd.read_csv(filepath_or_buffer="/content/drive/MyDrive/BeRep/Data.csv", usecols=['user_id', 'program']) #Load Dataframe
df.drop_duplicates(inplace=True) #drop duplicates
df = pd.crosstab(df["user_id"],df["program"]).reset_index(drop=True) #frequency table of factors
out = pd.DataFrame(columns=df.columns, index=df.columns) #create empty df
for i, j in itertools.combinations_with_replacement(df.columns, 2): # count subscribed users for each programs pair combination
    if i == j:
        out.loc[i,j] = df[i].sum()
    else:
        out.loc[i,j] = out.loc[j,i] = np.count_nonzero((df[i]!=0) & (df[j]!=0))
out.to_excel("/content/drive/MyDrive/BeRep/01-obtained_deliverable.xlsx") #Export Dataframe To Excel

expected_deliverable.png
![expected_deliverable.png](https://github.com/leobioinf0/BeRep/blob/main/expected_deliverable.png?raw=true)

obtained_deliverable.png
![obtained_deliverable.png](https://github.com/leobioinf0/BeRep/blob/main/obtained_deliverable.png?raw=true)

## Observed error

I have observed that to obtain a result identical to the expected one, it is necessary to commit an error.
The error consists in eliminating the duplicates without taking into account the `subscription_date` variable.

To explain what I mean I will load the data again, but this time I will keep and parse the variable `subscription_date`

In [4]:
#Patience! Parsing the dates can take a couple of minutes. :)
df = pd.read_csv(filepath_or_buffer="/content/drive/MyDrive/BeRep/Data.csv",parse_dates=["subscription_date"])

We can see that if we do not take into account the `subscription_date` variable, the data has `11994` duplicate entries.

In [5]:
print(df[['user_id', 'program']].duplicated().sum())

11994


If we carry out a brute elimination of these `11994` duplicates, the results we will obtain will be identical to the expected ones.

However, when we take `subscription_date` into account, the data only shows `25` duplicates 

In [6]:
print(df.duplicated().sum())

25


Accepting these premises we can make some assumptions:

Different types of duplicates can be interpreted in different ways. 
1. Full duplicated entries, containing exactly the same `subscription_date` (ie: same date and same time), are true duplicates, so one of them should be removed since two events cannot occur at the same time. For example: 



In [7]:
df.iloc[[622819,622927]]

,user_id,subscription_date,program
622819,ba98a4592609eb4fd7a7d6d1b0ce765b717dcf1f853b73...,2021-10-25 12:00:00,CXPI
622927,ba98a4592609eb4fd7a7d6d1b0ce765b717dcf1f853b73...,2021-10-25 12:00:00,CXPI


This must be a **true duplicate** since it doesn't make sense for a user to subscribe to a program twice at exactly the same day and time.

2. But duplicated entries with the same `user_id` and the same `program` but different `subscription_date` can be interpreted as first a "**subscribe**" event and second as an "**unsubscribe**" event. For example:



In [8]:
df.iloc[[43966,44404]]

,user_id,subscription_date,program
43966,61eb4abcc06b6a0afb496e52c36133db2fc10d36a5621e...,2021-12-03 01:00:00,OBS
44404,61eb4abcc06b6a0afb496e52c36133db2fc10d36a5621e...,2022-02-21 12:13:57,OBS


This can be interpreted as the user having subscribed to the `OBS` program on `2021-12-03 01:00:00` and then unsubscribed on `2022-02-21 12:13:57`. If this were the case, we would have to remove both records to get a true count of users who are subscribed to the programs.

3. Following this line of thought, we can say that if we found an even number of duplicate entries with the same `user_id` and the same `program` but a different `subscription_date`, the user's current status would be **"unsubscribed"** to the program. But if duplicate entries are found in odd number, the user's current state in the program would be **"subscribed"** so it should count as one, so only the last record should be kept. For example:

In [9]:
df[(df.user_id == "1f646282d4fe1a047097ef73940e8b9e682889836c6714c33b7bfdc8f74d3536")&(df.program == "OBS")].sort_values(by=["subscription_date"])

,user_id,subscription_date,program
1535463,1f646282d4fe1a047097ef73940e8b9e682889836c6714...,2019-09-30 23:05:10,OBS
1545290,1f646282d4fe1a047097ef73940e8b9e682889836c6714...,2021-09-15 09:22:20,OBS
1193661,1f646282d4fe1a047097ef73940e8b9e682889836c6714...,2021-12-01 08:47:11,OBS
1560467,1f646282d4fe1a047097ef73940e8b9e682889836c6714...,2021-12-15 08:38:07,OBS
1564101,1f646282d4fe1a047097ef73940e8b9e682889836c6714...,2022-01-19 08:59:23,OBS


This can be interpreted as:
1. subscribed on `2019-09-30 23:05:10`
2. unsubscribed on `2021-09-15 09:22:20`
3. subscribed on `2021-12-01 08:47:11`
4. unsubscribed on `2021-12-15 08:38:07`
5. and finally subscribed on `2022-01-19 08:59:23`

Finally, in my humble opinion, I think that doing a simple `drop_duplicates()` without taking into account the different cases would lead us to obtain incorrect results.